In [16]:
import openai, json

client = openai.OpenAI()
messages = []

In [17]:
def get_weather(city):
    return "33 degress celcius."

FUNCTION_MAP = {"get_weather": get_weather}

In [18]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city",
            "parameters": {
                "type": "object",
                "properties":{
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",   
                    },
                },
                "required": ["city"],
            },
        },
    },
]

In [19]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    
    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    } 
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            
            print(f"Calling function: {function_name} with : {arguments}")
            
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}
                
            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Run {function_name} with args{arguments} for a result of {result}")

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result,
            })
        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")

def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [20]:
while True:
    message = input("Send a message to the LLM...") 
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user","content": message})
        print(f"User: {message}")
        call_ai()

User: My name is Bitnoori
AI: Nice to meet you, Bitnoori! How can I assist you today?
User: what is weather in Seoul
Calling function: get_weather with : {"city":"Seoul"}
Run get_weather with args{'city': 'Seoul'} for a result of 33 degress celcius.
AI: The weather in Seoul is currently 33 degrees Celsius. If you need more details or have any other questions, feel free to ask!
User: quite
AI: It seems like you might find the weather quite warm! If you need any tips on staying cool or have other questions, just let me know!


In [22]:
messages

[{'role': 'user', 'content': 'My name is Bitnoori'},
 {'role': 'assistant',
  'content': 'Nice to meet you, Bitnoori! How can I assist you today?'},
 {'role': 'user', 'content': 'what is weather in Seoul'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_3gjHz2iyaX07TvuMyC6Vjm05',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Seoul"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_3gjHz2iyaX07TvuMyC6Vjm05',
  'name': 'get_weather',
  'content': '33 degress celcius.'},
 {'role': 'assistant',
  'content': 'The weather in Seoul is currently 33 degrees Celsius. If you need more details or have any other questions, feel free to ask!'},
 {'role': 'user', 'content': 'quite'},
 {'role': 'assistant',
  'content': 'It seems like you might find the weather quite warm! If you need any tips on staying cool or have other questions, just let me know!'}]